In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from tqdm import tqdm
from sklearn.metrics import roc_auc_score

import edward as ed
ed.set_seed(42)

import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

from edward.models import Normal, Categorical

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [3]:
N = 100   # number of images in a minibatch.
D = 784   # number of features.
K = 10    # number of classes.
neurons = 1000
layers = 1
epochs = 100
n_samples = 100
eval_batch_size = 1000

iters = int(mnist.train.num_examples/N*epochs)

In [4]:
X_val = mnist.validation.images
Y_val = np.argmax(mnist.validation.labels,axis=1)

In [7]:
def neural_network(X):
    h = tf.nn.relu(tf.matmul(X, W_0) + b_0)
    if layers == 2:
        h = tf.nn.relu(tf.matmul(h, W_1) + b_1)
    h = tf.matmul(h, W_2) + b_2
    return h

# MODEL
with tf.name_scope("model"):
    W_0 = Normal(loc=tf.zeros([D, neurons]), scale=tf.ones([D, neurons]), name="W_0")
    b_0 = Normal(loc=tf.zeros(neurons), scale=tf.ones(neurons), name="b_0")
    
    if layers == 2:
        W_1 = Normal(loc=tf.zeros([neurons, neurons]), scale=tf.ones([neurons, neurons]), name="W_1")
        b_1 = Normal(loc=tf.zeros(neurons), scale=tf.ones(neurons), name="b_1")
    
    W_2 = Normal(loc=tf.zeros([neurons, K]), scale=tf.ones([neurons, K]), name="W_2")
    b_2 = Normal(loc=tf.zeros(K), scale=tf.ones(K), name="b_2")

    X = tf.placeholder(tf.float32, [None, D], name="X")
    y = Categorical(neural_network(X), name="y")
        
# INFERENCE
with tf.name_scope("posterior"):
    with tf.name_scope("qW_0"):
        qW_0 = Normal(loc=tf.get_variable("qW0", shape=[D, neurons], initializer=tf.contrib.layers.xavier_initializer()),
                      scale=tf.nn.softplus(
                          tf.get_variable("qW0s", shape=[D, neurons], initializer=tf.contrib.layers.xavier_initializer())))
    with tf.name_scope("qb_0"):
        qb_0 = Normal(loc=tf.get_variable("qb0", shape=[neurons], initializer=tf.contrib.layers.xavier_initializer()),
                      scale=tf.nn.softplus(
                          tf.get_variable("qb0s", shape=[neurons], initializer=tf.contrib.layers.xavier_initializer())))
    
    if layers == 2:
        with tf.name_scope("qW_1"):
            qW_1 = Normal(loc=tf.get_variable("qW1", shape=[neurons, neurons], initializer=tf.contrib.layers.xavier_initializer()),
                          scale=tf.nn.softplus(
                              tf.get_variable("qW1s", shape=[neurons, neurons], initializer=tf.contrib.layers.xavier_initializer())))        
        with tf.name_scope("qb_1"):
            qb_1 = Normal(loc=tf.get_variable("qb1", shape=[neurons], initializer=tf.contrib.layers.xavier_initializer()),
                          scale=tf.nn.softplus(
                              tf.get_variable("qb1s", shape=[neurons], initializer=tf.contrib.layers.xavier_initializer())))
        
    with tf.name_scope("qW_2"):
        qW_2 = Normal(loc=tf.get_variable("qW2", shape=[neurons, K], initializer=tf.contrib.layers.xavier_initializer()),
                      scale=tf.nn.softplus(
                          tf.get_variable("qW2s", shape=[neurons, K], initializer=tf.contrib.layers.xavier_initializer())))
    with tf.name_scope("qb_2"):
        qb_2 = Normal(loc=tf.get_variable("qb2", shape=[K], initializer=tf.contrib.layers.xavier_initializer()),
                      scale=tf.nn.softplus(
                          tf.get_variable("qb2s", shape=[K], initializer=tf.contrib.layers.xavier_initializer())))

In [8]:
def eval_acc_auc(dataset):
    prob_lst = []
    pred_lst = []
    Y = []

    for _ in tqdm(range(dataset.num_examples//eval_batch_size)):
        X, y = mnist.test.next_batch(eval_batch_size, shuffle=False)
        prob_lst_temp = []
        for _ in range(n_samples):
            W0_samp = qW_0.sample()
            b0_samp = qb_0.sample()
            if layers == 2:
                W1_samp = qW_1.sample()
                b1_samp = qb_1.sample()
            W2_samp = qW_2.sample()
            b2_samp = qb_2.sample()

            h_samp = tf.nn.relu(tf.matmul(X, W0_samp) + b0_samp)
            if layers == 2:
                h_samp = tf.nn.relu(tf.matmul(h_samp, W1_samp) + b1_samp)
            h_samp = tf.matmul(h_samp, W2_samp) + b2_samp
            prob = tf.nn.softmax(h_samp)

            # Also compue the probabiliy of each class for each (w,b) sample.
            prob_lst_temp.append(prob.eval())
        pred_lst.append(np.argmax(np.mean(prob_lst_temp,axis=0),axis=1))
        prob_lst.append(np.max(np.mean(prob_lst_temp,axis=0),axis=1))
        Y.append(y)

    Y_pred = np.concatenate(pred_lst)
    Y_prob = np.concatenate(prob_lst)
    Y_actual = np.argmax(np.concatenate(Y), axis=1)

    acc = (Y_pred == Y_actual).mean()*100

    auc = roc_auc_score(Y_pred == Y_actual, Y_prob)

    return acc, auc

In [9]:
y_ph = tf.placeholder(tf.int32, [N])

if layers == 1:
    inference = ed.KLqp({W_0: qW_0, b_0: qb_0,
                         W_2: qW_2, b_2: qb_2}, data={y: y_ph})
elif layers == 2:
    inference = ed.KLqp({W_0: qW_0, b_0: qb_0,
                         W_1: qW_1, b_1: qb_1,
                         W_2: qW_2, b_2: qb_2}, data={y: y_ph})
    
inference.initialize(n_iter=iters, n_print=100, optimizer='adam', scale={y: float(mnist.train.num_examples) / N})

sess = tf.InteractiveSession()

tf.global_variables_initializer().run()

for i in range(1, inference.n_iter+1):
    X_batch, Y_batch = mnist.train.next_batch(N)
    Y_batch = np.argmax(Y_batch,axis=1)
    info_dict = inference.update(feed_dict={X: X_batch, y_ph: Y_batch})
    inference.print_progress(info_dict)
    #if i*N % mnist.train.num_examples == 0: # epoch
    #    print('  Val acc:', eval_acc(X_val, Y_val))

55000/55000 [100%] ██████████████████████████████ Elapsed: 292s | Loss: 46681.801


In [12]:
X_test = mnist.test.images
Y_test = np.argmax(mnist.test.labels,axis=1)
print('ADVI', neurons, layers, epochs, n_samples, eval_acc_auc(mnist.test))

  0%|          | 0/10 [00:00<?, ?it/s]

ResourceExhaustedError: OOM when allocating tensor with shape[1,784,1000]
	 [[Node: posterior/qW_0/Normal/sample_45/random_normal/RandomStandardNormal = RandomStandardNormal[T=DT_INT32, dtype=DT_FLOAT, seed=42, seed2=5025, _device="/job:localhost/replica:0/task:0/device:GPU:0"](posterior/qW_0/Normal/sample_45/concat)]]
	 [[Node: Softmax_44/_111 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_171_Softmax_44", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]

Caused by op 'posterior/qW_0/Normal/sample_45/random_normal/RandomStandardNormal', defined at:
  File "/usr/lib/python3.5/runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py", line